In [132]:
from lxml import etree
from pprint import pprint

In [140]:
with open('template.xml', 'r') as f:
    data = f.read()

root = etree.fromstring(data)
elements = root.xpath("//mxCell[string-length(@id) > 1 and string-length(@value) < 20]")


In [145]:
get_id = etree.XPath("@id")
get_value = etree.XPath("@value")
get_x = etree.XPath("mxGeometry/@x")
get_y = etree.XPath("mxGeometry/@y")
get_w = etree.XPath("mxGeometry/@width")
get_h = etree.XPath("mxGeometry/@height")
get_style = etree.XPath("@style")

def get_colour(el: etree.Element) -> str:
    style = get_style(el)[0]
    style_lst = style.split(";")
    for item in style_lst:
        if "=" in item:
            key, val = item.split("=")
            if key == "fillColor":
                return val
        
    return ""

In [146]:
def get_data(el: etree.Element) -> dict:
    width = int(get_w(el)[0])
    height = int(get_h(el)[0])
    size = width*height

    return {
            "id": get_id(el)[0],
            "value": get_value(el)[0].lower(),
            "x": int(get_x(el)[0]),
            "y": int(get_y(el)[0]),
            "w": width,
            "h": height,
            "size": size,
            "colour": get_colour(el),
        }

def rescale(lst: list, x: int, y: int, w: int, h: int) -> list:
    for i in range(len(lst)):
        lst[i]['x'] -= x
        lst[i]['y'] -= y
        lst[i]['w'] /= w
        lst[i]['h'] /= h

In [147]:
element_data = [get_data(el) for el in elements]
element_data = sorted(element_data, key=lambda x: x["size"], reverse=True)
canvas = element_data[0]
rescale(element_data, canvas['x'], canvas['y'], canvas['w'], canvas['h'])

IndexError: list index out of range

In [126]:
def contains(el1: dict,el2: dict) -> bool:
    if el1['x'] <= el2['x'] and el1['y'] <= el2['y']:
        if el2['x'] + el2['w'] <= el1['x'] + el1['w'] and el2['y'] + el2['h'] <= el1['y'] + el1['h']:
            return True
    return False


In [127]:
i = len(element_data)-2
while i > -1 and len(element_data) > 1:
    element_data[i]["children"] =  []
    for j in range(len(element_data)-1, i, -1):
        if contains(element_data[i], element_data[j]):
            element_data[i]["children"].append(element_data[j])
            element_data.pop(j)
    i -= 1


In [128]:
element_tree = element_data[0]

In [129]:
pprint(element_tree)

{'children': [{'children': [],
               'colour': '#ffe6cc',
               'h': 1.0,
               'id': 'GI5be8NEMBNBnBu12xkf-2',
               'size': 33600,
               'value': '',
               'w': 0.23076923076923078,
               'x': 0,
               'y': 0}],
 'colour': '#dae8fc',
 'h': 1.0,
 'id': 'GI5be8NEMBNBnBu12xkf-1',
 'size': 145600,
 'value': '',
 'w': 1.0,
 'x': 0,
 'y': 0}


In [104]:
import pandas as pd

In [105]:
test = pd.DataFrame(element_tree)

In [107]:
element_tree

{'id': 'GI5be8NEMBNBnBu12xkf-1',
 'value': '',
 'x': 120,
 'y': 120,
 'w': 520,
 'h': 280,
 'size': 145600,
 'colour': '#dae8fc',
 'children': [{'id': 'GI5be8NEMBNBnBu12xkf-3',
   'value': 'title',
   'x': 240,
   'y': 120,
   'w': 400,
   'h': 60,
   'size': 24000,
   'colour': '#fff2cc',
   'children': []},
  {'id': 'GI5be8NEMBNBnBu12xkf-2',
   'value': '',
   'x': 120,
   'y': 120,
   'w': 120,
   'h': 280,
   'size': 33600,
   'colour': '#ffe6cc',
   'children': [{'id': 'GI5be8NEMBNBnBu12xkf-6',
     'value': 'slider',
     'x': 130,
     'y': 250,
     'w': 100,
     'h': 20,
     'size': 2000,
     'colour': ''},
    {'id': 'GI5be8NEMBNBnBu12xkf-5',
     'value': 'slider',
     'x': 130,
     'y': 220,
     'w': 100,
     'h': 20,
     'size': 2000,
     'colour': '',
     'children': []},
    {'id': 'GI5be8NEMBNBnBu12xkf-4',
     'value': 'slider',
     'x': 130,
     'y': 190,
     'w': 100,
     'h': 20,
     'size': 2000,
     'colour': '',
     'children': []},
    {'id': 'G

In [106]:
test

,id,value,x,y,w,h,size,colour,children
0,GI5be8NEMBNBnBu12xkf-1,,120,120,520,280,145600,#dae8fc,"{'id': 'GI5be8NEMBNBnBu12xkf-3', 'value': 'tit..."
1,GI5be8NEMBNBnBu12xkf-1,,120,120,520,280,145600,#dae8fc,"{'id': 'GI5be8NEMBNBnBu12xkf-2', 'value': '', ..."
2,GI5be8NEMBNBnBu12xkf-1,,120,120,520,280,145600,#dae8fc,"{'id': 'GI5be8NEMBNBnBu12xkf-15', 'value': '',..."
